In [48]:
import pandas as pd
import csv
import numpy as np
f=open('./basket.csv')
data=list(csv.reader(f))
sizes={}
for item in data:
    if sizes.get(len(item)) is not None:
        sizes[len(item)]+=1
    else:
        sizes[len(item)]=1
mostSize=0
curPop=0
for size, pop in zip(sizes.keys(), sizes.values()):
    if curPop<pop:
        mostSize=size
        curPop=pop
data=[i[:4] for i in data if len(i)==mostSize]
print(data[:10])
df = pd.DataFrame(data)
display(df)


[['MBE', 'BLACK', 'Yorktown Heights', '10598'], ['MBE', 'BLACK', 'Long Beach', '11561'], ['MBE', 'ASIAN', 'Brooklyn', '11235'], ['MBE', 'ASIAN', 'New York', '10026'], ['MBE', 'BLACK', 'New York', '10026'], ['MBE', 'HISPANIC', 'New York', '10034'], ['MBE', 'ASIAN', 'New York', '10018'], ['MBE', 'BLACK', 'Jamaica', '11434'], ['WBE', 'NON-MINORITY', 'New York', '10022'], ['MBE', 'BLACK', 'Staten Island', '10304']]


,0,1,2,3
0,MBE,BLACK,Yorktown Heights,10598
1,MBE,BLACK,Long Beach,11561
2,MBE,ASIAN,Brooklyn,11235
3,MBE,ASIAN,New York,10026
4,MBE,BLACK,New York,10026
...,...,...,...,...
1099,MBE,BLACK,Paterson,7514
1100,WBE,NON-MINORITY,New York,10023
1101,MBE,ASIAN,Valley Stream,11580
1102,MBE,BLACK,Brooklyn,11214


In [21]:
from collections import Counter,OrderedDict
# This Function is used to generate all the subset of any set
def genSubsets(l):
    powerSetSize = 2 ** len(l)
    powerSet = []
    for i in range(1,powerSetSize):
        tempEle = []
        for j in range(len(l)):
            binFlagInd = i & (1 << j)
            if binFlagInd:
                tempEle.append(l[j])
        powerSet.append(tempEle)
    return powerSet


In [22]:
def genCandidate(Fk1): #Fk1 indicates F(k-1), it is a list of lists
    Ck = []
    k1  = len(Fk1[0])

    # COMBINE STEP
    for i in range(len(Fk1)-1):
        for j in range(i+1,len(Fk1)):
            f1,f2 = Fk1[i],Fk1[j]

            if f1[:len(f1)-1] == f2[:len(f2)-1] and f1[-1] < f2[-1]:
                tempC = f1 + [f2[-1]]

                # PRUNING STEP
                subset = genSubsets(tempC)
                appendSts = True
                for item in subset:
                    if len(item) == k1 and item not in Fk1:
                        appendSts = False
                if appendSts:
                    Ck.append(tempC)              
    return Ck

In [23]:
def searchInT(t,candid):
    found = True
    for eachCandid in candid:
        if eachCandid not in t:
            found = False
    return found

In [24]:
def initPass(txList): # list of transactions, most possibly a dict
    allTx = [item for tx in txList for item in tx]
    allTx.sort()
    cntr =  OrderedDict()
    for tx in allTx:
        cntr[tx] = cntr.get(tx,0) + 1 # 0 default value hai, agar tx ke liye get na hua then 0 output aayega get function ka

    return cntr

In [25]:
def apriori(T,minSup):
    finalSet = []
    c1 = initPass(T)
    f = [[item] for item in c1.keys() if c1[item]/len(T) >= minSup] # f1
    # print(f)
    for item in f:
        finalSet.append(item)

    while len(f) != 0:
        Ck = genCandidate(f)
        # print("Ck")
        # print(Ck)
        freqDict = {}
        for t in T:
            for candidate in Ck:
                #+1 whenver entire candidate occurs within the given transaction
                if searchInT(t,candidate):
                    freqDict[tuple(candidate)] = freqDict.get(tuple(candidate),0) + 1 #agar koi candidate ke sabhi item given transaction me hue to +1 ho jayega freqDict[tuple(candidate)] ka
        # print("freqDict")
        # print(freqDict)
        f = []
        for c in freqDict.keys():
            if freqDict[c]/len(T)>= minSup:# /len(T) because formula for support is freq(candidate)/no. of total transactions. freq from freqDict calculated above
                f.append(list(c))

        # print("f")
        # print(f)
        if len(f) != 0:
            f = sorted(f,key=lambda x : (len(x),*x))
            for item in f:
                finalSet.append(item)
    # print(finalSet)
    return finalSet




In [26]:
def generate_rules(frequent_itemsets, transactions, min_conf):
    rules = []
    N = len(transactions)

    support_count = {}
    for itemset in frequent_itemsets:
        count = 0
        for t in transactions:
            if all(i in t for i in itemset):
                count += 1
        support_count[tuple(itemset)] = count / N
    # print(support_count)
    for itemset in frequent_itemsets:
        if len(itemset) < 2:
            continue  

        subsets = genSubsets(itemset)
        for antecedent in subsets:
            if len(antecedent) == 0 or len(antecedent) == len(itemset):
                continue
            consequent = list(set(itemset) - set(antecedent))

            antecedent_support = support_count.get(tuple(sorted(antecedent)), 0)
            if antecedent_support == 0:
                continue

            rule_support = support_count.get(tuple(sorted(itemset)), 0)
            confidence = rule_support / antecedent_support

            if confidence >= min_conf:
                rules.append((antecedent, consequent, confidence))

    return rules

In [67]:
def printRule(F=None, rules=None, minConfidence=None, minSupport=None):
        if F is None:
            for ant, cons, conf in rules:
                print(f"{ant} => {cons} (confidence = {conf:.2f}) size = {len(ant)+len(cons)}")
            return
        print(f'\n Item-sets with support above {minSupport}')
        print(F)
        print(f"\nRules with confidence ≥ {minConfidence}:")
        for ant, cons, conf in rules:
            print(f"{ant} => {cons} (confidence = {conf:.2f}) size = {len(ant)+len(cons)}")
        print()
        print()
        print()

#### 1. Running on different support and confidence values.

In [76]:
resultData=[]
for minConfidence in np.arange(0.05, 1.0, 0.05):
    for minSupport in np.arange(0.05, 1.0, 0.05):
        F=apriori(data, minSup=minSupport)
        # if len(F)==0:
        #     continue
        rules = generate_rules(F, data, min_conf=minConfidence)
        # if len(rules)==0:
        #     continue
        resultData.append({'apriori':F, 'rules':rules, 'minSupport': minSupport, 'minConfidence':minConfidence})
        # printRule(F, rules, minSupport=minSupport, minConfidence=minConfidence)
for item in resultData:
    printRule(item['apriori'], item['rules'], item['minConfidence'], item['minSupport'])



 Item-sets with support above 0.05
[['ASIAN'], ['BLACK'], ['Bronx'], ['Brooklyn'], ['HISPANIC'], ['MBE'], ['NON-MINORITY'], ['New York'], ['WBE'], ['ASIAN', 'MBE'], ['ASIAN', 'New York'], ['BLACK', 'Brooklyn'], ['BLACK', 'MBE'], ['Bronx', 'MBE'], ['Brooklyn', 'MBE'], ['HISPANIC', 'MBE'], ['MBE', 'New York'], ['NON-MINORITY', 'New York'], ['NON-MINORITY', 'WBE'], ['New York', 'WBE'], ['ASIAN', 'MBE', 'New York'], ['BLACK', 'Brooklyn', 'MBE'], ['NON-MINORITY', 'New York', 'WBE']]

Rules with confidence ≥ 0.05:
['ASIAN'] => ['MBE'] (confidence = 0.99) size = 2
['MBE'] => ['ASIAN'] (confidence = 0.32) size = 2
['ASIAN'] => ['New York'] (confidence = 0.28) size = 2
['New York'] => ['ASIAN'] (confidence = 0.18) size = 2
['BLACK'] => ['Brooklyn'] (confidence = 0.22) size = 2
['Brooklyn'] => ['BLACK'] (confidence = 0.42) size = 2
['BLACK'] => ['MBE'] (confidence = 1.00) size = 2
['MBE'] => ['BLACK'] (confidence = 0.44) size = 2
['Bronx'] => ['MBE'] (confidence = 0.91) size = 2
['MBE'] => ['Br

#### 2. What is maximum size of rule that can be created?

In [77]:
ruleSizeStore=[]
curMaxSize=0
for item in resultData:
    rules=item['rules']
    tempStore=[]
    tempSize=0
    for rule in rules:
        size=len(rule[0])+len(rule[1]) #antecedent+ consequent
        if size>tempSize:
            tempStore.clear()
            tempStore.append(rule)
            tempSize=size
        else:
            tempStore.append(rule)
    if tempSize>curMaxSize:
        ruleSizeStore=tempStore.copy()
        curMaxSize=tempSize
    elif tempSize==curMaxSize:
        ruleSizeStore.extend(tempStore)
printRule(rules=ruleSizeStore)

['ASIAN'] => ['New York', 'MBE'] (confidence = 0.28) size = 3
['MBE'] => ['New York', 'ASIAN'] (confidence = 0.09) size = 3
['ASIAN', 'MBE'] => ['New York'] (confidence = 0.28) size = 3
['New York'] => ['ASIAN', 'MBE'] (confidence = 0.18) size = 3
['ASIAN', 'New York'] => ['MBE'] (confidence = 0.98) size = 3
['MBE', 'New York'] => ['ASIAN'] (confidence = 0.37) size = 3
['BLACK'] => ['Brooklyn', 'MBE'] (confidence = 0.22) size = 3
['Brooklyn'] => ['BLACK', 'MBE'] (confidence = 0.42) size = 3
['BLACK', 'Brooklyn'] => ['MBE'] (confidence = 1.00) size = 3
['MBE'] => ['Brooklyn', 'BLACK'] (confidence = 0.10) size = 3
['BLACK', 'MBE'] => ['Brooklyn'] (confidence = 0.22) size = 3
['Brooklyn', 'MBE'] => ['BLACK'] (confidence = 0.60) size = 3
['NON-MINORITY'] => ['WBE', 'New York'] (confidence = 0.40) size = 3
['New York'] => ['NON-MINORITY', 'WBE'] (confidence = 0.50) size = 3
['NON-MINORITY', 'New York'] => ['WBE'] (confidence = 1.00) size = 3
['WBE'] => ['NON-MINORITY', 'New York'] (confiden

#### 3. At what Confidence value, Minimum number of rules are generated.

In [81]:

minRulesStore = {}

for item in resultData:
    conf = item['minConfidence']
    resultCount = len(item['rules'])

    if conf not in minRulesStore:
        minRulesStore[conf] = resultCount
    else:
        if resultCount < minRulesStore[conf]:
            minRulesStore[conf] = resultCount
minConf = min(minRulesStore, key=minRulesStore.get)
minRules = minRulesStore[minConf]

print("Confidence Value -> Minimum Rules Generated (at some support level):")
for conf, count in sorted(minRulesStore.items()):
    print(f"Confidence = {conf:.2f} → Min Rules = {count}")

print(f"\n Minimum number of rules ({minRules}) are generated at confidence = {minConf:.2f}")


Confidence Value -> Minimum Rules Generated (at some support level):
Confidence = 0.05 → Min Rules = 0
Confidence = 0.10 → Min Rules = 0
Confidence = 0.15 → Min Rules = 0
Confidence = 0.20 → Min Rules = 0
Confidence = 0.25 → Min Rules = 0
Confidence = 0.30 → Min Rules = 0
Confidence = 0.35 → Min Rules = 0
Confidence = 0.40 → Min Rules = 0
Confidence = 0.45 → Min Rules = 0
Confidence = 0.50 → Min Rules = 0
Confidence = 0.55 → Min Rules = 0
Confidence = 0.60 → Min Rules = 0
Confidence = 0.65 → Min Rules = 0
Confidence = 0.70 → Min Rules = 0
Confidence = 0.75 → Min Rules = 0
Confidence = 0.80 → Min Rules = 0
Confidence = 0.85 → Min Rules = 0
Confidence = 0.90 → Min Rules = 0
Confidence = 0.95 → Min Rules = 0

 Minimum number of rules (0) are generated at confidence = 0.05
